# MATERIAS

In [ ]:
# Cada clave es un semestre (columna). El orden sigue la foto.

semestres = {
    1: [
        "Pre Cálculo",
        "Cálculo 1",
        "Matemáticas Discretas",
        "Introducción a la Ciencia de Datos",
        "Programación 1",
        "Instituciones Políticas",
        "Pensamiento Crítico en Ciencia de Datos 1",
        "Idioma 1",
    ],
    2: [
        "Cálculo 2",
        "Álgebra Lineal",
        "Probabilidad",
        "Programación 2",
        "Ética en Ciencia de Datos",
        "Fundamento de Estadística",
        "Idioma 2",
    ],
    3: [
        "Inferencia Estadística",
        "Data Mining",
        "Estructuras de Datos",
        "Introducción a la Economía",
        "Seguridad y Privacidad",
        "Idioma 3",
    ],
    4: [
        "Estadística Bayesiana",
        "Machine Learning 1",
        "Bases de Datos",
        "Disciplina 1",
        "Pensamiento Crítico en Ciencia de Datos 2",
        "Idioma 4",
    ],
    5: [
        "Visualización 1",
        "Métodos Numéricos",
        "Machine Learning 2",
        "Big Data",
        "Disciplina 2",
        "Emprendimiento",
        "Idioma 5",
    ],
    6: [
        "Visualización 2",
        "Optimización",
        "Deep Learning",
        "Formulación de Proyectos",
        "Disciplina 3",
        "Idioma 6",
    ],
    7: [
        "Prácticas",
    ],
    8: [
        "Taller de Habilidades Profesionales",
        "Ciberseguridad",
        "Proyectos",
        "Disciplina 4",
        "Taller de Habilidades Gerenciales",
    ],
}



# CLASE PROYECCIÓN

In [2]:
import pandas as pd

# =============================================================
# DICCIONARIO DE PRERREQUISITOS
# =============================================================
# Se define a partir del PDF de materias. Cada materia tiene como valor una lista
# con los nombres de las materias que se deben haber aprobado antes.
prerrequisitos = {
    "Idioma 2": ["Idioma 1"],
    "Cálculo 2": ["Cálculo 1"],
    "Probabilidad": ["Introducción a la Ciencia de Datos"],
    "Programación 2": ["Programación 1"],
    "Álgebra Lineal": ["Matemáticas Discretas"],
    "Idioma 3": ["Idioma 2"],
    "Data Mining": ["Probabilidad", "Programación 1"],
    "Inferencia Estadística": ["Probabilidad"],
    "Estructuras de Datos": ["Programación 2"],
    "Bases de Datos": ["Estructuras de Datos"],
    "Machine Learning 1": ["Data Mining"],
    "Estadística Bayesiana": ["Inferencia Estadística", "Programación 2", "Cálculo 2"],
    "Pensamiento Crítico en Ciencia de Datos 2": ["Pensamiento Crítico en Ciencia de Datos 1"],
    "Big Data": ["Bases de Datos", "Data Mining"],
    "Visualización 1": ["Introducción a la Ciencia de Datos"],
    "Machine Learning 2": ["Machine Learning 1"],
    "Métodos Numéricos": ["Álgebra Lineal", "Cálculo 2", "Programación 1"],
    "Visualización 2": ["Visualización 1", "Inferencia Estadística", "Bases de Datos"],
    "Optimización": ["Métodos Numéricos"],
    "Deep Learning": ["Machine Learning 2"],
    "Disciplina 2": ["Disciplina 1"],
    "Disciplina 3": ["Disciplina 2"],
    "Disciplina 4": ["Disciplina 3"],
    "Idioma 4": ["Idioma 3"],
    "Idioma 5": ["Idioma 4"],
    "Idioma 6": ["Idioma 5"],
    "Ciberseguridad": ["Estructuras de Datos"],
    "Formulación de Proyectos": ["Pensamiento Crítico en Ciencia de Datos 2"],
    "Taller de Habilidades Gerenciales": ["Emprendimiento"],
    "Proyectos": ["Formulación de Proyectos"],
}

# =============================================================
# DATOS DE EJEMPLO DEL SEMESTRE ACTUAL

# Incluye las columnas:
# - "materia": nombre del curso
# - "estudiantes_sem_actual": número actual de estudiantes matriculados
# - "porcentaje_perdida": proporción esperada de deserción o pérdida
data = {
    "materia": [
        "Cálculo 2", "Probabilidad", "Programación 2", "Álgebra Lineal", "Idioma 2"
    ],
    "estudiantes_sem_actual": [100, 95, 98, 90, 110],
    "porcentaje_perdida": [0.1, 0.15, 0.05, 0.08, 0.12]  # proporciones (10%, 15%, etc.)
}
df = pd.DataFrame(data)

# =============================================================
# FUNCIÓN PARA PROYECTAR ESTUDIANTES AL SIGUIENTE SEMESTRE

def proyectar_estudiantes(df, prerrequisitos, df_previos):
    # Se crea una nueva columna para la proyección del siguiente semestre
    df["proyeccion_siguiente"] = 0

    # Itera fila por fila en la tabla actual
    for i, row in df.iterrows():
        materia = row["materia"]
        perdida = row["porcentaje_perdida"]

        # Obtenemos los prerrequisitos de la materia actual si existen
        prereqs = prerrequisitos.get(materia, [])

        if prereqs:
            # Si tiene prerrequisitos, calculamos el promedio de estudiantes aprobados
            # de las materias prerrequisito (del semestre anterior)
            aprobados_prereq = []
            for p in prereqs:
                if p in df_previos.index:
                    # Estudiantes aprobados = estudiantes del semestre anterior * (1 - pérdida)
                    aprobados_prereq.append(
                        df_previos.loc[p, "estudiantes_sem_actual"] * (1 - df_previos.loc[p, "porcentaje_perdida"])
                    )

            # Si encontramos al menos un prerrequisito con datos, tomamos el promedio
            if aprobados_prereq:
                proyeccion = sum(aprobados_prereq) / len(aprobados_prereq)
            else:
                # Si no hay datos de prerrequisitos, usamos los estudiantes actuales
                proyeccion = row["estudiantes_sem_actual"]

        else:
            # Si no tiene prerrequisitos, se asume continuidad del mismo número de estudiantes
            proyeccion = row["estudiantes_sem_actual"]

        # Finalmente, aplica el porcentaje de pérdida del curso actual
        df.at[i, "proyeccion_siguiente"] = proyeccion * (1 - perdida)

    return df

# =============================================================
# DATOS DEL SEMESTRE ANTERIOR (EJEMPLO)

# En este caso, se simula el primer semestre
df_previos = pd.DataFrame({
    "materia": ["Cálculo 1", "Introducción a la Ciencia de Datos", "Programación 1", "Matemáticas Discretas", "Idioma 1"],
    "estudiantes_sem_actual": [120, 115, 118, 110, 130],
    "porcentaje_perdida": [0.1, 0.12, 0.08, 0.07, 0.1]
}).set_index("materia")

df_proyeccion = proyectar_estudiantes(df, prerrequisitos, df_previos)




/tmp/ipython-input-112575318.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '97.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, "proyeccion_siguiente"] = proyeccion * (1 - perdida)


In [3]:
df_proyeccion

,materia,estudiantes_sem_actual,porcentaje_perdida,proyeccion_siguiente
0,Cálculo 2,100,0.10,97.200
1,Probabilidad,95,0.15,86.020
2,Programación 2,98,0.05,103.132
3,Álgebra Lineal,90,0.08,94.116
4,Idioma 2,110,0.12,102.960


Se necesita ajustar tabla para aplicar los porcentajes de perdida